In [1]:
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser
from api_saver import ApiSaver
from FUNCTIONS import get_current_location

In [2]:
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

Current Latitude: 44.4048, Longitude: 8.9444


In [3]:
from route_df import RouteDF
routeDF = RouteDF()

from api_route import ApiRoute
current_position = {'longitude': 130.844161,
                    'latitude': -12.456694}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 2000.0 # in meters
number_sites = 10

route = ApiRoute(routeDF.route_data)

# route_df = route.cut_route_data(current_position)
route_df = route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# route_df = route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)

Nearest point index: 8
    longitude   latitude  altitudeSmoothed  inclinationSmoothed   cumDistance  \
0  130.844161 -12.456694         16.350559            -0.024998    407.095561   
1  130.849361 -12.440116          8.870473             0.009980   2407.095561   
2  130.864829 -12.432468          7.344984             0.002014   4407.095561   
3  130.883247 -12.432408         19.917411             0.000566   6407.095561   
4  130.900256 -12.437241         19.535214             0.032636   8407.095561   
5  130.916674 -12.444541         14.082252             0.006061  10407.095561   
6  130.932244 -12.451610         17.790217            -0.000259  12407.095561   
7  130.949308 -12.458322         16.597118             0.003234  14407.095561   
8  130.966263 -12.463929         36.186164            -0.006110  16407.095561   
9  130.983291 -12.469279         34.239105             0.008118  18407.095561   

   maxSpeed       theta  cumDistanceCut  
0        50   44.136411             0.0  
1

In [4]:
api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser)
api_executer = ApiExecuter(requester=api_requester)

sites_df = api_requester.forecast_sites

Response status from siteinfo: OK.
Response from siteinfo is empty.
Current sites' info has been retrieved. 
 Empty DataFrame
Columns: [name, longitude, latitude, altitude]
Index: []


In [5]:
# api_requester.get_site_info(print_is_requested=True)

In [6]:
api_executer.add_sites(route_df, print_is_requested=True)

Response status from siteadd: OK.
Site with name 0 has been added. 
        name  longitude  latitude  altitude
585533    0    130.844  -12.4567      32.0
Response status from siteadd: OK.
Site with name 1 has been added. 
        name  longitude  latitude  altitude
585533    0    130.844  -12.4567      32.0
585534    1    130.849  -12.4401      16.0
Response status from siteadd: OK.
Site with name 2 has been added. 
        name  longitude  latitude  altitude
585533    0    130.844  -12.4567      32.0
585534    1    130.849  -12.4401      16.0
585535    2    130.865  -12.4325      14.0
Response status from siteadd: OK.
Site with name 3 has been added. 
        name  longitude  latitude  altitude
585533    0    130.844  -12.4567      32.0
585534    1    130.849  -12.4401      16.0
585535    2    130.865  -12.4325      14.0
585536    3    130.883  -12.4324      24.0
Response status from siteadd: OK.
Site with name 4 has been added. 
        name  longitude  latitude  altitude
585533    

In [7]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [8]:
# position = {
#     'latitude': -13.0,
#     'longitude': 134.0
# }
# api_requester.get_site_edit(585396, True, name="aa")
# api_requester.get_site_edit(585395, True, position=position)
# api_requester.get_site_edit(585396, True, name="df", position=position)

In [9]:
forecast_df = api_requester.get_solar_forecast()
# forecast_df = api_requester.get_solar_forecast_cloudmove()
print(forecast_df)

Response status from getforecast: OK.
Solar forecast have been retrieved.
                              gk  gh_max  gh      tt    ff  rr      dd     rh  \
site_id time                                                                    
585513  2023-09-07 12:00:00  0.0     0.0   0  25.400  10.0   0  274.00  69.00   
        2023-09-07 12:15:00  0.0     0.0   0  25.275  10.5   0  275.25  70.25   
        2023-09-07 12:30:00  0.0     0.0   0  25.150  11.0   0  276.50  71.50   
        2023-09-07 12:45:00  0.0     0.0   0  25.025  11.5   0  277.75  72.75   
        2023-09-07 13:00:00  0.0     0.0   0  24.900  12.0   0  279.00  74.00   
...                          ...     ...  ..     ...   ...  ..     ...    ...   
585532  2023-09-16 10:00:00  0.0     0.0   0  27.600  13.0   0  341.00  54.00   
        2023-09-16 10:15:00  0.0     0.0   0  27.325  12.5   0  334.50  55.75   
        2023-09-16 10:30:00  0.0     0.0   0  27.050  12.0   0  328.00  57.50   
        2023-09-16 10:45:00  0.0   

In [10]:
api_saver = ApiSaver()

# api_saver._data_restructure(route_df, sites_df, forecast_df)
raw_data_forecast = api_saver.save_raw_data(route_df, sites_df, forecast_df)

ValueError: The index of the forecast dataframe and the sites dataframe are not the same

In [ ]:
from forecast_preprocessor import ForecastPreprocessor
forecast_preprocessor = ForecastPreprocessor()
hours_in_advance = 5
forecast_preprocessor.forecast_preprocessing(route_df, raw_data_forecast, hours_in_advance, print_is_requested=False)